# Школа глубокого обучения

<a href="https://mipt.ru/science/labs/laboratoriya-neyronnykh-sistem-i-glubokogo-obucheniya/"><img align="right" src="https://avatars1.githubusercontent.com/u/29918795?v=4&s=200" alt="DeepHackLab" style="position:relative;top:-40px;right:10px;height:100px;" /></a>



### Физтех-Школа Прикладной математики и информатики МФТИ 
### Лаборатория нейронных сетей и глубокого обучения (DeepHackLab)
*Дедлайн -- 25 ноября.*

*Захаркин Илья (ФИВТ МФТИ), Хахулин Тарас (ФРТК МФТИ)*

### Перед началом выполнения задания настоятельно рекомендуется посмотреть [лекцию](https://vk.com/doc83865491_454180587?hash=37e49000cda3837459&dl=cf3df2e170d9d05351) и прорешать задания в [ноутбуке с семинара](https://github.com/deepmipt/dlschl/blob/master/materials/%5Blesson3%5Dperceptron/%5Bsem3%5DPerceptron_seminar.ipynb).

## Домашнее задание 4
### Реализация перцептрона и нейрона с сигмоидальной функцией активации

**Поздравляем! :) **  
Это Ваше первое домашнее задание на нейронные сети, ведь один нейрон - это неотъемлимая составляющая любой глубокой сети, которые на сегодняший день используются повсеместно (и которые Вам предстоит изучить далее по курсу). 

В данном задании Вам нужно будет: 
- самостоятельно реализовать класс **`Perceptron()`** с пороговой функцией активации
- обучить и протестировать Ваш нейрон на сгенерированных и реальных данных (файлы с реальными данными помещены в папку /data в этой же директории)
- сравнить качество работы Вашего класса с классом из библиотеки `scikit-learn` (`sklearn.linear_model.Perceptron()`)
- самостоятельно реализовать класс Neuron() с сигмоидной функцией активации и самостоятельно протестировать его

### Небольшое введение

<img src="http://blog.radario.ru/wp-content/uploads/2017/05/model.png" style="width:750px;height:300px"/>

(на картинке получается, что "алгоритм" и "модель" - разные вещи, но в машинном обучении их обычно считают синонимами, то есть "алгоритм" = "модель")

Почти любой алгоритм машинного обучения, решающий задачу *классификации* или *регрессии*, работает так:

1. (*стадия инициализации*) Задаются его **гиперпараметры**, то есть те величины, которые не "выучиваются" алгоритмом в процессе обучения самостоятельно 
2. (*стадия обучения*) Алгоритм запускается на данных, **обучаясь** на них и меняя свои **параметры** (не путать с *гипер*параметрами) каким-то определённым образом (например, с помощью *метода градиентного спуска*), исходя из функции потерь (её называют *loss function*). Функция потерь, по сути, говорит, где и как ошибается модель
3.  (*стадия предсказания*) Модель готова, и теперь с помощью неё можно делать **предсказания** на новых объектах

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap  # тут лежат разные штуки для цветовой магии
import numpy as np
import pandas as pd

### Задача 1 (3 балла)
#### Класс Perceptron

В даном разделе будет решаться задача **бинарной классификации** с помощью перцептрона:  
- *Входные данные*: матрица $X$ размера $(n, m)$ и столбец $y$ из нулей и единиц размера $(n, 1)$. Строкам матрицы соответствуют объекты, столбцам - признаки (то есть строка $i$ есть набор признаков (*признаковое описание*) объекта $x_i$).
- *Выходные данные*: столбец $\hat{y}$ из нулей и единиц размера $(n, 1)$ - предсказания алгоритма.

Краткая справка по перцептрону (было на занятии):

Перцептрон взвешивает входы с определёнными весами и выдаёт результат в виде пороговой функции от взвешенной суммы:
![title](./perceptron.png)

Чтобы понять, как мы будем обновлять параметры модели (веса), нужно знать, какую функцию потерь мы оптимизируем (находим минимум). В данном случае мы решаем задачу бинарной классификации (2 класса: 1 или 0), возьмём в качестве функции потерь среднеквадратичную ошибку:
$$J(w, x) = \frac{1}{2n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{2n}\sum_{i=1}^{n} (f(w \cdot x_i) - y_i)^2$$  
Здесь $w \cdot x_i$ - скалярное произведение, а $f(w \cdot x_i)$ - пороговая функция: 
$$
f(z) =
\begin{cases}
1, &\text{если } w \cdot x_i > 0 \\
0, &\text{если } w \cdot x_i \le 0
\end{cases}
$$  

**Примечание:** Здесь предполагается, что $b$ - свободный член - является частью вектора весов: $w_0$. Тогда, если к $X$ приписать слева единичный столбец, в скалярном произведении $b$ будет именно как свободный член.

** Реализуйте функцию потерь $J$ (0.5 балла): **

In [ ]:
def J(y_pred, y):
    return # Ваш код здесь

Поскольку у *пороговой функции* не существует производной (Вы её график видели? Выглядит он, конечно, простым, но производная таких не любит), то мы не можем использовать градиентный спуск, ведь:
$$ \frac{\partial J}{\partial w} = \frac{1}{n} (f(w \cdot X) - y)f^{'}(w \cdot X)X$$ 
где $f^{'}(w \cdot X)$ - посчитать не получится. Но ведь хочется как-то обновлять веса, иначе как обучить алгоритм отличать груши от яблок? Поэтому предлагается взять $w = w - \alpha\Delta{w}$, где $\Delta{w} = \frac{1}{n}X^T(\hat{y} - y)$ (не забудьте, что при $w_0 = b$ признак $x_0$ = 1). Это правило является неким частным случаем градиентного спуска для данного случая.

Теперь, вооружившись всеми формулами и силой духа, нужно написать свой класс **`Perceptron()`**. Уже есть код класса и немного кода реализации. По-максимуму используйте **Numpy** при реализации, т.к. будет проверяться и скорость работы Вашего алгоритма.

In [ ]:
class Perceptron:
    def __init__(self, w=None, b=0):
        """
        :param: w -- вектор весов
        :param: b -- смещение
        """
        # Пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
        
    def activate(self, x):
        return x > 0
        
    def forward_pass(self, X):
        """
        Рассчитывает ответ перцептрона при предъявлении набора объектов
        :param: X -- матрица примеров размера (n, m), каждая строка - отдельный объект
        :return: вектор размера (n, 1) из нулей и единиц с ответами перцептрона 
        """
        n = X.shape[0]
        y_pred = np.zeros((n, 1))  # y_predicted - предсказанные классы
        # Ваш код здесь
        return y_pred
    
    def backward_pass(self, X, y, y_pred, learning_rate=0.005):
        """
        Обновляет значения весов перцептрона в соответствии с этим объектом
        :param: X -- матрица входов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                learning_rate - "скорость обучения" (символ alpha в формулах выше)
        В этом методе ничего возвращать не нужно
        """
        # Ваш код здесь
    
    def fit(self, X, y, num_epochs=300):
        """
        Спускаемся в минимум
        :param: X -- матрица входов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                num_epochs -- количество итераций обучения
        :return: J_values -- вектор значений функции потерь
        """
        self.w = np.zeros((X.shape[1], 1))  # столбец (m, 1)
        self.b = 0  # смещение
        J_values = []  # значения функции потерь на различных итерациях обновления весов
        
        for i in range(num_epochs):
            # Ваш код здесь

        return J_values

* Класс готов. Посмотрим, правильно ли ведёт себя Ваш перцептрон. Далее идут несколько ячеек с тестовым кодом, Вам нужно просто запустить их и проверить, чтобы результаты запуска совпадали с соответствующими числами из таблиц:

**forward_pass() (0.5 балла)**

In [ ]:
w = np.array([1., 2.]).reshape(2, 1)
b = 2.
X = np.array([[1., 2., -1.], [3., 4., -3.2]])

perceptron = Perceptron(w, b)
y_pred = perceptron.forward_pass(X.T)
print ("y_pred = " + str(y_pred))

|Должно быть||
|------|-------|
|**y_pred**|[1, 1, 0]|

**backward_pass() (0.5 балла)**

In [ ]:
y = np.array([1, 0, 1]).reshape(3, 1)

In [ ]:
perceptron.backward_pass(X.T, y, y_pred)

print ("w = " + str(perceptron.w))
print ("b = " + str(perceptron.b))

|Должно быть||
|-|-|
|**w**| [[ 0.995], [1.988]] |
|**b**| 2.0 |

* Посмотрим, как меняется функция потерь в течение процесса обучения на реальных данных - датасет "Яблоки и Груши" из лекции:

In [ ]:
data = pd.read_csv("./data/apples_pears.csv")

In [ ]:
data.head()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=data['target'], cmap='rainbow')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('симметричность', fontsize=14)
plt.ylabel('желтизна', fontsize=14)
plt.show();

**Вопрос:** Какой класс соответствует яблокам (какого они цвета на графике)?

**Ответ:** <Ваш ответ>

Обозначим, что здесь признаки, а что - классы:

In [ ]:
X = data.iloc[:,:2].values  # матрица объекты-признаки
y = data['target'].values.reshape((-1, 1))  # классы (столбец из нулей и единиц)

** Вывод функции потерь (1.5 балла) **  
Функция потерь должна убывать и в итоге стать близкой к 0

In [ ]:
%%time
perceptron = # Ваш код здесь
J_values = # Ваш код здесь

plt.figure(figsize=(10, 8))
plt.plot(J_values)
plt.title('Функция потерь', fontsize=15)
plt.xlabel('номер итерации', fontsize=14)
plt.ylabel('$J(\hat{y}, y)$', fontsize=14)
plt.show()

Посмотрим, как перцептрон классифицировал объекты из выборки:

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(data.iloc[:, 0], data.iloc[:, 1], c=perceptron.forward_pass(X), cmap='spring')
plt.title('Яблоки и груши', fontsize=15)
plt.xlabel('симметричность', fontsize=14)
plt.ylabel('желтизна', fontsize=14)
plt.show();

### Задача 2 (2 балла)

В этой задаче нужно сравнить качество работы Вашего перцептрона и алгоритма из библиотеки `sklearn` на датасете с сайта [Kaggle](https://www.kaggle.com) - [Gender Recognition by Voice](https://www.kaggle.com/primaryobjects/voicegender). В данном датасете в качестве признаков выступают различные звуковые характеристики голоса, а в качестве классов - пол (мужчина/женщина). Подробнее о самих признаках можно почитать [на странице датасета](https://www.kaggle.com/primaryobjects/voicegender) (на английском). Нашей целью пока что является просто протестировать на этих данных два алгоритма.

**! Обратите внимание на имя функции из sklearn - skPerceptron** (это сделано, чтобы не совпадало с именем Вашего класса)

In [ ]:
import pandas as pd
from sklearn.linear_model import Perceptron as skPerceptron
from sklearn.metrics import accuracy_score

In [ ]:
data_path = './data/voice.csv'
data = pd.read_csv(data_path)
data['label'] = data['label'].apply(lambda x: 1 if x == 'male' else 0)

In [ ]:
data.head()

In [ ]:
# Чтобы перемешать данные. Изначально там сначала идут все мужчины, потом все женщины
data = data.sample(frac=1)

In [ ]:
X_train = data.iloc[:int(len(data)*0.7), :-1]  # матрица объекты-признаки
y_train = data.iloc[:int(len(data)*0.7), -1]  # истинные значения пола (мужчина/женщина)

X_test = data.iloc[int(len(data)*0.7):, :-1]  # матрица объекты-признаки
y_test = data.iloc[int(len(data)*0.7):, -1]  # истинные значения пола (мужчина/женщина)

Тут нужно натренировать Ваш перцептрон и перцептрон из `sklearn` на этих данных:

In [ ]:
# Ваш код здесь

Сравним доли правильных ответов:

In [ ]:
print('Точность (доля правильных ответов, из 100%) моего перцептрона: {:d}'.format(accuracy_score(<Ваш код здесь>) * 100))
print('Точность (доля правильных ответов) перцептрона из sklearn: {:.1f} %'.format(accuracy_score(<Ваш код здесь>) * 100))

** Вопрос:** Хорошее ли качество показывает перцептрон? Как Вы думаете, почему? Можете писать любые мысли на этот счёт.

**Ответ:**

### Задача 3 (5 балла)

#### Нейрон (сигмоидальный) (2 балла)

В данном случае мы снова решаем задачу бинарной классификации (2 класса: 1 или 0), но здесь уже будет другая функция активации:
$$J(w, x) = \frac{1}{2n}\sum_{i=1}^{n} (\hat{y_i} - y_i)^2 = \frac{1}{2n}\sum_{i=1}^{n} (\sigma(w \cdot x_i) - y_i)^2$$  
Здесь $w \cdot x_i$ - скалярное произведение, а $\sigma(w \cdot x_i) =\frac{1}{1+e^{-w \cdot x_i}} $ - сигмоида.  

**Примечание:** Здесь предполагается, что $b$ - свободный член - является частью вектора весов и является $w_0$. Тогда, если к $X$ приписать единичный столбец, в скалярном произведении $b$ будет именно как свободный член.

Формула для обновления весов при градиентном спуске будут такие:

$$ \frac{\partial J}{\partial w} = \frac{1}{n}\sum_{i=1}^{n} (\sigma(w \cdot x_i) - y_i)\sigma(w \cdot x_i)(1 - \sigma(w \cdot x_i))x_i$$ (не забудьте, что $x_0$ = 1)

Функции с семинара:

In [ ]:
def sigmoid(x):
    """Сигмоидальная функция"""
    pass

In [ ]:
def sigmoid_prime(x):
    """Производная сигмоиды"""
    pass

Теперь нужно написать нейрон с сигмоидной функцией активации. Здесь всё похоже на перцептрон, но будут по-другому обновляться веса и другая функция активации:

In [ ]:
class Neuron:
    def __init__(self, w=None, b=0):
        """
        :param: w -- вектор весов
        :param: b -- смещение
        """
        # Пока что мы не знаем размер матрицы X, а значит не знаем, сколько будет весов
        self.w = w
        self.b = b
        
    def activate(self, x):
        return # Ваш код здесь
        
    def forward_pass(self, X):
        """
        Рассчитывает ответ перцептрона при предъявлении набора объектов
        :param: X -- матрица примеров размера (n, m), каждая строка - отдельный объект
        :return: вектор размера (n, 1) из нулей и единиц с ответами перцептрона 
        """
        n = X.shape[0]
        y_pred = np.zeros((n, 1))  # y_predicted - предсказанные классы
        # Ваш код здесь
        return y_pred
    
    def backward_pass(self, X, y, y_pred, learning_rate=0.005):
        """
        Обновляет значения весов перцептрона в соответствии с этим объектом
        :param: X -- матрица входов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                learning_rate - "скорость обучения" (символ alpha в формулах выше)
        В этом методе ничего возвращать не нужно
        """
        # Ваш код здесь
    
    def fit(self, X, y, num_epochs=300):
        """
        Спускаемся в минимум
        :param: X -- матрица входов размера (n, m)
                y -- вектор правильных ответов размера (n, 1)
                num_epochs -- количество итераций обучения
        :return: J_values -- вектор значений функции потерь
        """
        self.w = np.zeros((X.shape[1], 1))  # столбец (m, 1)
        self.b = 0  # смещение
        J_values = []  # значения функции потерь на различных итерациях обновления весов
        
        for i in range(num_epochs):
            # Ваш код здесь
        
        return J_values

**Тестирование нейрона (3 балла)**

Здесь Вам нужно самим протестировать новый нейрон **на тех же данных** по аналогии с тем, как это было проделано с перцептроном.  (можете смело копировать код, только будьте осторожны - кое-что в нём всё же скорее всего придётся поправить).
В итоге нужно вывести: 
* график, на котором будет показано, как изменяется функция потерь $J$ в зависимости от числа итераций обучения (1.5 балла)
* график с раскраской выборки сигмоидальным нейроном (1.5 балла)

In [ ]:
# Ваш код здесь (можете использовать много ячеек)